In [7]:
import pandas as pd
import requests
import time

# Load the dataset
spar_df = pd.read_csv("spar_zurich_final.csv")

In [9]:
spar_df["type"] = "Spar"


In [10]:
spar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    48 non-null     object 
 1   street  48 non-null     object 
 2   city    48 non-null     object 
 3   lat     48 non-null     float64
 4   lng     48 non-null     float64
 5   type    48 non-null     object 
dtypes: float64(2), object(4)
memory usage: 2.4+ KB


In [11]:
city_to_district = {
    "Altikon": "Winterthur", "Adliswil": "Horgen", "Aesch": "Dietikon", "Aeugst am Albis": "Affoltern",
    "Affoltern am Albis": "Affoltern", "Andelfingen": "Andelfingen", "Bachenbülach": "Bülach",
    "Bachs": "Dielsdorf", "Bäretswil": "Hinwil", "Bassersdorf": "Bülach", "Bauma": "Pfäffikon",
    "Benken": "Andelfingen", "Berg am Irchel": "Andelfingen", "Birmensdorf": "Dietikon",
    "Bonstetten": "Affoltern", "Boppelsen": "Dielsdorf", "Brütten": "Winterthur", "Bubikon": "Hinwil",
    "Buch am Irchel": "Andelfingen", "Buchs": "Dielsdorf", "Bülach": "Bülach", "Dachsen": "Andelfingen",
    "Dägerlen": "Winterthur", "Dällikon": "Dielsdorf", "Dänikon": "Dielsdorf", "Dättlikon": "Winterthur",
    "Dielsdorf": "Dielsdorf", "Dietikon": "Dietikon", "Dietlikon": "Bülach", "Dinhard": "Winterthur",
    "Dorf": "Andelfingen", "Dübendorf": "Uster", "Dürnten": "Hinwil", "Egg": "Uster", "Eglisau": "Bülach",
    "Elgg": "Winterthur", "Ellikon an der Thur": "Winterthur", "Elsau": "Winterthur", "Embrach": "Bülach",
    "Erlenbach": "Meilen", "Fällanden": "Uster", "Fehraltorf": "Pfäffikon", "Feuerthalen": "Andelfingen",
    "Fischenthal": "Hinwil", "Flaach": "Andelfingen", "Flurlingen": "Andelfingen",
    "Freienstein-Teufen": "Bülach", "Geroldswil": "Dietikon", "Glattfelden": "Bülach",
    "Gossau": "Hinwil", "Greifensee": "Uster", "Grüningen": "Hinwil", "Hagenbuch": "Winterthur",
    "Hausen am Albis": "Affoltern", "Hedingen": "Affoltern", "Henggart": "Andelfingen",
    "Herrliberg": "Meilen", "Hettlingen": "Winterthur", "Hinwil": "Hinwil", "Hittnau": "Pfäffikon",
    "Hochfelden": "Bülach", "Hombrechtikon": "Meilen", "Horgen": "Horgen", "Höri": "Bülach",
    "Hüntwangen": "Bülach", "Hüttikon": "Dielsdorf", "Illnau-Effretikon": "Pfäffikon",
    "Kappel am Albis": "Affoltern", "Kilchberg": "Horgen", "Kleinandelfingen": "Andelfingen",
    "Kloten": "Bülach", "Knonau": "Affoltern", "Küsnacht": "Meilen", "Langnau am Albis": "Horgen",
    "Laufen-Uhwiesen": "Andelfingen", "Lindau": "Pfäffikon", "Lufingen": "Bülach", "Männedorf": "Meilen",
    "Marthalen": "Andelfingen", "Maschwanden": "Affoltern", "Maur": "Uster", "Meilen": "Meilen",
    "Mettmenstetten": "Affoltern", "Mönchaltorf": "Uster", "Neerach": "Dielsdorf",
    "Neftenbach": "Winterthur", "Niederglatt": "Dielsdorf", "Niederhasli": "Dielsdorf",
    "Niederweningen": "Dielsdorf", "Nürensdorf": "Bülach", "Oberembrach": "Bülach",
    "Oberengstringen": "Dietikon", "Oberglatt": "Dielsdorf", "Oberrieden": "Horgen",
    "Oberweningen": "Dielsdorf", "Obfelden": "Affoltern", "Oetwil am See": "Meilen",
    "Oetwil an der Limmat": "Dietikon", "Opfikon": "Bülach", "Ossingen": "Andelfingen",
    "Otelfingen": "Dielsdorf", "Ottenbach": "Affoltern", "Pfäffikon": "Pfäffikon",
    "Pfungen": "Winterthur", "Rafz": "Bülach", "Regensberg": "Dielsdorf", "Regensdorf": "Dielsdorf",
    "Rheinau": "Andelfingen", "Richterswil": "Horgen", "Rickenbach": "Winterthur",
    "Rifferswil": "Affoltern", "Rorbas": "Bülach", "Rümlang": "Dielsdorf", "Rüschlikon": "Horgen",
    "Russikon": "Pfäffikon", "Rüti": "Hinwil", "Schlatt": "Winterthur", "Schleinikon": "Dielsdorf",
    "Schlieren": "Dietikon", "Schöfflisdorf": "Dielsdorf", "Schwerzenbach": "Uster",
    "Seegräben": "Hinwil", "Seuzach": "Winterthur", "Stadel": "Dielsdorf", "Stäfa": "Meilen",
    "Stallikon": "Affoltern", "Stammheim": "Andelfingen", "Steinmaur": "Dielsdorf",
    "Thalheim an der Thur": "Andelfingen", "Thalwil": "Horgen", "Trüllikon": "Andelfingen",
    "Truttikon": "Andelfingen", "Turbenthal": "Winterthur", "Uetikon am See": "Meilen",
    "Uitikon": "Dietikon", "Unterengstringen": "Dietikon", "Urdorf": "Dietikon", "Uster": "Uster",
    "Volken": "Andelfingen", "Volketswil": "Uster", "Wädenswil": "Horgen", "Wald": "Hinwil",
    "Wallisellen": "Bülach", "Wangen-Brüttisellen": "Uster", "Wasterkingen": "Bülach",
    "Weiach": "Dielsdorf", "Weiningen": "Dietikon", "Weisslingen": "Pfäffikon",
    "Wettswil am Albis": "Affoltern", "Wetzikon": "Hinwil", "Wiesendangen": "Winterthur",
    "Wil": "Bülach", "Wila": "Pfäffikon", "Wildberg": "Pfäffikon", "Winkel": "Bülach",
    "Winterthur": "Winterthur", "Zell": "Winterthur", "Zollikon": "Meilen", "Zumikon": "Meilen",
    "Zürich": "Zürich"
}

In [14]:
spar_df['district'] = spar_df['city'].map(city_to_district)


In [15]:
spar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      48 non-null     object 
 1   street    48 non-null     object 
 2   city      48 non-null     object 
 3   lat       48 non-null     float64
 4   lng       48 non-null     float64
 5   type      48 non-null     object 
 6   district  47 non-null     object 
dtypes: float64(2), object(5)
memory usage: 2.8+ KB


In [16]:
spar_df.to_csv("spar_zurich_final_dist.csv", index=False)


In [17]:
import requests
import time

api_key = "API_KEY"  # Replace with your actual API key

def get_place_id(name, street, city):
    query = f"{name}, {street}, {city}, Zurich, Switzerland"
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": query,
        "inputtype": "textquery",
        "fields": "place_id",
        "key": api_key
    }
    response = requests.get(url, params=params)
    time.sleep(1)
    result = response.json()
    if result.get("candidates"):
        return result["candidates"][0]["place_id"]
    return None

def get_place_details(place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "name,rating,user_ratings_total,formatted_address,opening_hours",
        "key": api_key
    }
    response = requests.get(url, params=params)
    time.sleep(1)
    return response.json().get("result", {})

# Drop duplicates by location (optional but recommended)
spar_df = spar_df.drop_duplicates(subset=["name", "street", "city"]).reset_index(drop=True)

# Get place IDs
spar_df['place_id'] = spar_df.apply(lambda row: get_place_id(row['name'], row['street'], row['city']), axis=1)

# Get place details
spar_df['place_details'] = spar_df['place_id'].apply(lambda pid: get_place_details(pid) if pid else None)

# Extract rating and reviews
spar_df['rating'] = spar_df['place_details'].apply(lambda x: x.get("rating") if x else None)
spar_df['reviews'] = spar_df['place_details'].apply(lambda x: x.get("user_ratings_total") if x else None)



In [18]:
spar_df.to_csv("spar_zurich_enriched.csv", index=False)

In [2]:
import pandas as pd

# Load both datasets
supermarkets_df = pd.read_csv("supermarkets_with_ratings.csv")
spar_df = pd.read_csv("spar_zurich_enriched.csv")

# Combine them
merged_df = pd.concat([supermarkets_df, spar_df], ignore_index=True)

# Drop any exact duplicates if necessary
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

# Save the final merged dataset
merged_df.to_csv("all_supermarkets_enriched.csv", index=False)


In [3]:
def classify_size(reviews):
    if pd.isna(reviews):
        return "Unknown"
    elif reviews <= 300:
        return "Small"
    elif reviews <= 1000:
        return "Medium"
    else:
        return "Large"

# Apply to your DataFrame
merged_df['size_category'] = merged_df['reviews'].apply(classify_size)


In [4]:
merged_df.head()

,name,lat,lng,street,city,type,district,place_id,place_details,rating,reviews,size_category
0,Migros-Supermarkt - Affoltern a. A.,47.278438,8.453841,Oberdorfstrasse 4,Affoltern am Albis,Migros,Affoltern,ChIJiUOy2zkGkEcRy22nDwkqAks,"{'formatted_address': 'Oberdorfstrasse 4, 8910...",4.2,702.0,Medium
1,Coop Supermarkt Affoltern am Albis,47.273871,8.445676,Büelstrasse 15,Affoltern am Albis,Coop,Affoltern,ChIJk6WFSkcGkEcRwsvlTASqby8,"{'formatted_address': 'Büelstrasse 15, 8910 Af...",4.1,1370.0,Large
2,Coop Supermarkt Obfelden,47.266924,8.424225,Ottenbacherstrasse 23,Obfelden,Coop,Affoltern,ChIJR20XCHAGkEcRC3aZj3d3dHE,"{'formatted_address': 'Ottenbacherstrasse 23, ...",4.0,284.0,Small
3,ALDI SUISSE,47.272743,8.447018,Industriestrasse 5,Affoltern am Albis,Aldi,Affoltern,ChIJLzM1Oh8HkEcRwBmfpSnSZx8,"{'formatted_address': 'Industriestrasse 5, 891...",4.1,384.0,Medium
4,Coop Pronto Shop mit Tankstelle Affoltern am A...,47.284026,8.453483,Im Wängli 6,Affoltern am Albis,Coop,Affoltern,ChIJ8XTvszoGkEcR_9pLF7bYgxU,"{'formatted_address': 'Im Wängli 6, 8910 Affol...",4.2,237.0,Small


In [5]:
merged_df.to_csv("all_supermarkets_with_size.csv", index=False)

In [8]:
def classify_rating(rating):
    if pd.isna(rating):
        return "Unknown"
    elif rating < 4.0:
        return "Below Avg"
    elif rating < 4.5:
        return "Average"
    else:
        return "Good"

merged_df['rating_category'] = merged_df['rating'].apply(classify_rating)



In [9]:
merged_df.to_csv("all_supermarkets_with_size_and_rating.csv", index=False)
